# **SUPERVISED LEARNING**

## 1. EDA

### **Data Overview**

In [ ]:
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
import seaborn as sns

import warnings
warnings.filterwarnings('ignore')

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
path = '/content/drive/MyDrive/'

In [ ]:
df = pd.read_excel(path+"churn_prediction.xlsx")
print(df)

In [ ]:
print(df.info())

### **Handle Missing/Duplicate Values**

In [ ]:
print(df.isna().sum())

In [ ]:
print(df.describe())

In [ ]:
# Tenure, HourSpendOnApp, CouponUsed, DaySinceLastOrder have 0 in values => replace 0
df['Tenure'].fillna(0, inplace=True)
df['HourSpendOnApp'].fillna(0, inplace=True)
df['CouponUsed'].fillna(0, inplace=True)
df['DaySinceLastOrder'].fillna(0, inplace=True)

In [ ]:
# OrderAmountHikeFromlastYear is Percentage increases in order from last year
# If the number orders last year = the number orders in this year => can have 0 in values => replace 0
df['OrderAmountHikeFromlastYear'].fillna(0, inplace=True)

# OrderCount is Total number of orders has been places in last month
# If the customer did not buy anything last month => can have 0 in values => replace 0
df['OrderCount'].fillna(0, inplace=True)

In [ ]:
# WarehouseToHome  => don't have 0 in values
# check outlier
sns.boxplot(df['WarehouseToHome'])


In [ ]:
# WarehouseToHome can cover outliers => replace median
WarehouseToHome_median = df['WarehouseToHome'].median()
df['WarehouseToHome'].fillna(WarehouseToHome_median, inplace=True)

print(df.isna().sum())
df.duplicated().any()

In [ ]:
df.duplicated().any()

### **Check imbalanced**

In [ ]:
label_ratio = df['Churn'].value_counts(normalize=True)
label_ratio

### **Univariate Analysis**

In [ ]:
df.dtypes.unique()

In [ ]:
obj_cols = df.select_dtypes(include=['object'])
num_cols = df.select_dtypes(exclude=['object'])

In [ ]:
num_cols.nunique()

In [ ]:
df['CityTier'] = df['CityTier'].astype(object)

print(df.info())

## 2. Transforms Features

In [ ]:
list_columns = ['PreferredLoginDevice','CityTier','PreferredPaymentMode','Gender','PreferedOrderCat','MaritalStatus']
data_encoded = pd.get_dummies(df, columns=list_columns, dtype='int')
data_encoded = data_encoded.drop(columns=['CustomerID'])

print(data_encoded.head())

## 3. Apply Random Forest model

### **Split train/test set & Normalization**

In [ ]:
from sklearn.model_selection import train_test_split
x=data_encoded.drop('Churn', axis = 1)
y=data_encoded[['Churn']]


x_train, x_test, y_train, y_test = train_test_split(x,y, test_size=0.3, random_state=42)

print(f"Number data of train set: {len(x_train)}")
print(f"Number data of test set: {len(x_test)}")

In [ ]:
from sklearn.preprocessing import MinMaxScaler

#Scale Feature:
scaler = MinMaxScaler()
scaler.fit(x_train)

x_train_scaled = scaler.transform(x_train)
x_test_scaled = scaler.transform(x_test)

### **Random Forest Classifier**

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, confusion_matrix
# Train a Random Forest classifier
classifier = RandomForestClassifier(random_state=42)
classifier.fit(x_train_scaled, y_train)

# Make predictions
y_pred = classifier.predict(x_test_scaled)

# Evaluate the model
RandomForest_accuracy = accuracy_score(y_test, y_pred)
RandomForest_conf_matrix = confusion_matrix(y_test, y_pred)

print(f'Accuracy: {RandomForest_accuracy:.2f}')
print(f'Confusion Matrix:\n{RandomForest_conf_matrix}')


## 4. Show Feature Importance from model

In [ ]:
clf_rand = RandomForestClassifier(max_depth=2, random_state=0)

clf_rand.fit(x_train_scaled, y_train)
y_ranf_pre_train = clf_rand.predict(x_train_scaled)
y_ranf_pre_test = clf_rand.predict(x_test_scaled)

In [ ]:
feats = {} # a dict to hold feature_name: feature_importance
for feature, importance in zip(x_test.columns, clf_rand.feature_importances_):
    feats[feature] = importance #add the name/value pair

importances = pd.DataFrame.from_dict(feats, orient='index').rename(columns={0: 'Gini-importance'})
importances = importances.sort_values(by='Gini-importance', ascending=True)

importances = importances.reset_index()

# Create bar chart
plt.figure(figsize=(10, 10))
plt.barh(importances.tail(20)['index'][:20], importances.tail(20)['Gini-importance'])

plt.title('Feature Important')

# Show plot
plt.show()

## 5. Choose top n features to analyse

In [ ]:
# Get CustomerID and Feature Important
Feature_Important = data_encoded[['Churn','Tenure', 'Complain', 'DaySinceLastOrder', 'CashbackAmount', 'MaritalStatus_Single', 'PreferedOrderCat_Mobile Phone']]

print(Feature_Important)

In [ ]:
# Get the churning users
Feature_Important = Feature_Important[Feature_Important['Churn']==1]

In [ ]:
Feature_Important = Feature_Important.drop(columns=['Churn'])
print(Feature_Important)

# **UNSUPERVISED LEARNING**

## 1. Dimension Reduction

In [ ]:
from sklearn.decomposition import PCA
pca = PCA(n_components=3)
pca.fit(Feature_Important)
PCA_ds = pd.DataFrame(pca.transform(Feature_Important), columns=(["col1","col2", "col3"]))
PCA_ds

In [ ]:
pca.explained_variance_ratio_

## 2. Apply K-Means model

### **Choosing K:**

In [ ]:
from sklearn.cluster import KMeans
ss = []
max_clusters = 10
for i in range(1, max_clusters+1):
    kmeans = KMeans(n_clusters=i, init='k-means++', random_state=42)
    kmeans.fit(PCA_ds)
    # Inertia method returns WCSS for that model
    ss.append(kmeans.inertia_)

# Plot the Elbow method
plt.figure(figsize=(10,5))
plt.plot(range(1, max_clusters+1), ss, marker='o', linestyle='--')
plt.title('Elbow Method')
plt.xlabel('Number of clusters')
plt.ylabel('WCSS')
plt.show()

### **Apply K-Means:**

In [ ]:
kmeans = KMeans(n_clusters=4, init='k-means++', random_state=42)
predicted_labels = kmeans.fit_predict(PCA_ds)

PCA_ds['clusters']=predicted_labels
Feature_Important['clusters']=predicted_labels

In [ ]:
Feature_Important

### **Silhouette Score:**

In [ ]:
from sklearn.metrics import silhouette_score

sil_score = silhouette_score(PCA_ds, predicted_labels)
print(sil_score)

### **Distribution of clusters:**

In [ ]:
pal = ["#682F2F","#B9C0C9", "#9F8A78","#F3AB60"]
pl = sns.countplot(x=Feature_Important["clusters"], palette= pal)
pl.set_title("Distribution Of The Clusters")
plt.show()

In [ ]:
pl = sns.scatterplot(data = Feature_Important,x=Feature_Important["Tenure"], y=Feature_Important["CashbackAmount"],hue=Feature_Important["clusters"], palette= pal)
pl.set_title("Cluster's Profile Based On CashbackAmount And Tenure")
plt.legend()
plt.show()

In [ ]:
pl = sns.scatterplot(data = Feature_Important,x=Feature_Important["DaySinceLastOrder"], y=Feature_Important["CashbackAmount"],hue=Feature_Important["clusters"], palette= pal)
pl.set_title("Cluster's Profile Based On CashbackAmount And DaySinceLastOrder")
plt.legend()
plt.show()

In [ ]:
pl = sns.countplot(data = Feature_Important,x=Feature_Important["MaritalStatus_Single"],hue=Feature_Important["clusters"], palette= pal)
pl.set_title("Cluster's Profile Based On MaritalStatus")
plt.legend()
plt.show()

In [ ]:
pl = sns.countplot(data = Feature_Important,x=Feature_Important["Complain"],hue=Feature_Important["clusters"], palette= pal)
pl.set_title("Cluster's Profile Based On Complain")
plt.legend()
plt.show()

In [ ]:
pl = sns.countplot(data = Feature_Important,x=Feature_Important["PreferedOrderCat_Mobile Phone"],hue=Feature_Important["clusters"], palette= pal)
pl.set_title("Cluster's Profile Based On PreferedOrderCat")
plt.legend()
plt.show()

## 3. GridSearchCV model

### **Split train/test set & Normalization**

In [ ]:
x2 = Feature_Important.drop('clusters', axis = 1)
y2 = Feature_Important[['clusters']]


x2_train, x2_test, y2_train, y2_test = train_test_split(x2,y2, test_size=0.3, random_state=42)

print(f"Number data of train set: {len(x2_train)}")
print(f"Number data of test set: {len(x2_test)}")

In [ ]:
from sklearn.preprocessing import MinMaxScaler

#Scale Feature:
scaler = MinMaxScaler()
scaler.fit(x2_train)

x2_train_scaled = scaler.transform(x2_train)
x2_test_scaled = scaler.transform(x2_test)

### **Apply Model**

In [ ]:
from sklearn.model_selection import GridSearchCV
param_grid = {
    'n_estimators': [10, 50, 100, 200],
    'max_depth': [None, 10, 20, 30],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'bootstrap': [True, False]
}

# Use GridSearchCV to find the best parameters
grid_search = GridSearchCV(clf_rand, param_grid, cv=5, scoring='balanced_accuracy')

# Fit the model
grid_search.fit(x2_train_scaled, y2_train)

# Print the best parameters
print("Best Parameters: ", grid_search.best_params_)

# Evaluate the best model on the test set
best_clf = grid_search.best_estimator_
accuracy = best_clf.score(x2_test_scaled, y2_test)
print("Test set accuracy: ", accuracy)

### **Feature Importance of Clusters**

In [ ]:
clf_rand.fit(x2_train_scaled, y2_train)
y2_ranf_pre_train = clf_rand.predict(x2_train_scaled)
y2_ranf_pre_test = clf_rand.predict(x2_test_scaled)

In [ ]:
feats = {} # a dict to hold feature_name: feature_importance
for feature, importance in zip(x2_test.columns, clf_rand.feature_importances_):
    feats[feature] = importance #add the name/value pair

importances = pd.DataFrame.from_dict(feats, orient='index').rename(columns={0: 'Gini-importance'})
importances = importances.sort_values(by='Gini-importance', ascending=True)

importances = importances.reset_index()

# Create bar chart
plt.figure(figsize=(10, 10))
plt.barh(importances.tail(20)['index'][:20], importances.tail(20)['Gini-importance'])

plt.title('Feature Important')

# Show plot
plt.show()